# Data Wrangling Template

## Gather

In [1]:
# import libraries
import pandas as pd
import numpy as np
import requests
import tweepy
import json

In [2]:
# read twitter-archive-enhanced.csv from directory to DataFrame
df = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# read image-predictions.tsv from Udacity's servers
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)
open(url.split('/')[-1], mode='wb').write(r.content);

# load tsv into DataFrame
df_ip = pd.read_csv(url.split('/')[-1], sep='\t')

In [4]:
# access twitter API via Tweepy library
consumer_key = 'VFSVvtWc0nLEn4zFIcSand0lh'
consumer_secret = 'hG6iRr8NdjKKNYlmwu0P6MI4dXG4Hrui0f7HpwgePE1VQ2oOnQ'
access_token = '19207319-hCgZ3LJ6iMjBvT8bx7H84DhILeALxPq9wj51DY2e9'
access_secret = 'PS8072UnIug2wJteMD7MawAGZN9s1taOC1gHxVnLrN8mQ'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# save all tweets to a txt file
id_errors = []

for tweet_id in list(df['tweet_id']):
    try:
        with open('tweet_json.txt', 'w') as file:
            status = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(status._json, file)
            file.write('\n')
    except:
        id_errors.append(tweet_id)

Rate limit reached. Sleeping for: 301
Rate limit reached. Sleeping for: 637


## Assess

In [ ]:
df.head()

In [ ]:
df_ip.head()

## Clean

#### Define

#### Code

#### Test